In [26]:
import numpy as np
import pandas as pd

#df = pd.DataFrame({'a': np.random.randn(1000), 'b': np.random.randn(1000),'N': np.random.randint(100, 1000, (1000)), 'x': 'x'})
df = pd.DataFrame({'a': np.random.uniform(size=int(1e3)), 'b':np.random.uniform(size=int(1e3)), 'N': np.random.randint(100, 1000, (1000)), 'x': 'x'})


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
N    1000 non-null int64
a    1000 non-null float64
b    1000 non-null float64
x    1000 non-null object
dtypes: float64(2), int64(1), object(1)
memory usage: 31.3+ KB


In [4]:
%timeit [row[1] * 2 for row in df.itertuples()]
%timeit [row[1] * 2 for row in df.itertuples(name=None)] # regular tuples

1.63 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
623 µs ± 2.22 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
df.tail()

,N,a,b,x
995,776,0.264615,-1.471044,x
996,471,-0.638369,0.888374,x
997,776,1.481771,0.331942,x
998,196,1.607678,0.979898,x
999,206,-0.850160,-1.025625,x


### simple moving average

In [7]:
%timeit df['MA'] = df['a'].rolling(window=5).mean()

341 µs ± 2.51 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
df.tail()

,N,a,b,x,MA
995,776,0.264615,-1.471044,x,0.010475
996,471,-0.638369,0.888374,x,0.003074
997,776,1.481771,0.331942,x,0.027953
998,196,1.607678,0.979898,x,0.043494
999,206,-0.850160,-1.025625,x,0.028213


In [11]:
%timeit df['MA'] = df['a'].rolling(window=90).mean()

348 µs ± 14.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [2]:
pdf = pd.read_csv('/home/starq/REP/DATA/FINANCE/Quotazioni/NVDA.csv' \
                                 ,usecols=['Date','Open', 'High', 'Low', 'Close', 'Volume'] \
                                 ,parse_dates=['Date'],infer_datetime_format=True \
                                 ,index_col='Date')


In [3]:
%timeit pdf['sma_20'] = pdf['Close'].rolling(window=20).mean()

The slowest run took 38.47 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 478 µs per loop


In [4]:
pdf['sma_20'] = pdf['Close'].rolling(window=20).mean()

In [10]:
pdf.tail()

,Open,High,Low,Close,Volume,sma_20,ema_20,pct_change
Date,,,,,,,,
2016-08-08,59.000000,59.139999,58.119999,58.740002,7626600,55.2490,55.051421,0.009278
2016-08-09,59.029999,59.459999,58.740002,58.889999,9308600,55.5535,55.417000,0.002554
2016-08-10,59.410000,59.500000,58.320000,58.509998,9393500,55.8400,55.711571,-0.006453
2016-08-11,58.820000,59.950001,58.700001,59.700001,17026600,56.1590,56.091422,0.020338
2016-08-12,61.900002,63.380001,60.630001,63.040001,35596500,56.6760,56.753191,0.055946


### exponential moving average

In [7]:
%timeit pdf['ema_20'] = pdf['Close'].ewm(span=20, adjust=False).mean()

The slowest run took 6.67 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 697 µs per loop


### pct_change()

In [9]:
%timeit pdf['pct_change'] = pdf['Close'].pct_change()

The slowest run took 114.11 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 708 µs per loop


### eval() 
https://stackoverflow.com/questions/38725355/when-to-use-dataframe-eval-versus-pandas-eval-or-python-eval

In [39]:
#import pandas as pd
#import numpy as np
#import numexpr
#import timeit

someDf = pd.DataFrame({'a':np.random.uniform(size=int(1e6)), 'b':np.random.uniform(size=int(1e6))})
someDf.info()
%timeit -n1000 someDf.eval("a**b - a*b > b**a - b/a") # DataFrame.eval() on notional expression
%timeit -n1000 eval("someDf['a']**someDf['b'] - someDf['a']*someDf['b'] > someDf['b']**someDf['a'] - someDf['b']/someDf['a']")
%timeit -n1000 pd.eval("someDf.a**someDf.b - someDf.a*someDf.b > someDf.b**someDf.a - someDf.b/someDf.a")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
a    1000000 non-null float64
b    1000000 non-null float64
dtypes: float64(2)
memory usage: 15.3 MB
1000 loops, best of 3: 12.5 ms per loop
1000 loops, best of 3: 15.7 ms per loop
1000 loops, best of 3: 10.5 ms per loop


In [17]:
someDf.tail()

,a,b
9995,0.976698,0.386643
9996,0.633809,0.423762
9997,0.778236,0.661393
9998,0.037200,0.906537
9999,0.270631,0.389314


In [40]:
pdf = pd.read_csv('/home/starq/REP/DATA/FINANCE/Quotazioni/NVDA.csv' \
                                 ,usecols=['Date','Open', 'High', 'Low', 'Close', 'Volume'] \
                                 ,parse_dates=['Date'],infer_datetime_format=True \
                                 ,index_col='Date')


In [45]:
import numexpr
'''
%timeit -n1000 df.eval("a**b - a*b > b**a - b/a") # DataFrame.eval() on notional expression
%timeit -n1000 eval("df['a']**df['b'] - df['a']*df['b'] > df['b']**df['a'] - df['b']/df['a']")
%timeit -n1000 pd.eval("df.a**df.b - df.a*df.b > df.b**df.a - df.b/df.a")
'''
%timeit -n500 pdf['perc_daily_range'] = pd.eval("((pdf.Low - pdf.High) / pdf.High) * 100")
%timeit -n500 pdf['perc_daily_range'] = pdf.eval("((Low - High) / High) * 100")
%timeit -n500 pdf['perc_daily_range'] = eval("((pdf['Low'] - pdf['High']) / pdf['High']) * 100")

500 loops, best of 3: 2.19 ms per loop
500 loops, best of 3: 2.42 ms per loop
500 loops, best of 3: 502 µs per loop


In [43]:
pdf.tail()

,Open,High,Low,Close,Volume,perc_daily_range
Date,,,,,,
2016-08-08,59.000000,59.139999,58.119999,58.740002,7626600,-1.724721
2016-08-09,59.029999,59.459999,58.740002,58.889999,9308600,-1.210893
2016-08-10,59.410000,59.500000,58.320000,58.509998,9393500,-1.983193
2016-08-11,58.820000,59.950001,58.700001,59.700001,17026600,-2.085071
2016-08-12,61.900002,63.380001,60.630001,63.040001,35596500,-4.338908
